# Equal-Weight S&P 500 Index Fund
#### Accepts value of portfolio and returns Excel file containing how many shares of each S&P 500 constituent incorporates an equal-weight version of the index fund.

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
stocks = pd.read_csv('CSV_files/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [55]:
IEX_CLOUD_API_TOKEN = 'pk_682c76a7979e43349b214ad64d6d41b7'

In [56]:
symbol = 'AAPL'
api_url = f'https://arbitrage.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = (requests.get(api_url).json())[0]
print(data)

{'avgTotalVolume': 56888460, 'calculationPrice': 'close', 'change': 0.49, 'changePercent': 0.00282, 'close': 174.49, 'closeSource': 'official', 'closeTime': 1692388800378, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 174.49, 'delayedPriceTime': 1692388799923, 'extendedChange': 0.21, 'extendedChangePercent': 0.0012, 'extendedPrice': 174.7, 'extendedPriceTime': 1692403196716, 'high': 175.1, 'highSource': '15 minute delayed price', 'highTime': 1692388799923, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 174.54, 'iexBidSize': 100, 'iexClose': 174.49, 'iexCloseTime': 1692388797614, 'iexLastUpdated': 1692388797614, 'iexMarketPercent': 0.015124807612614564, 'iexOpen': 172.06, 'iexOpenTime': 1692365400253, 'iexRealtimePrice': 174.49, 'iexRealtimeSize': 200, 'iexVolume': 925217, 'lastTradeTime': 1692388799923, 'latestPrice': 174.49, 'latestSource': 'Close', 'latestTime': 'August 18, 2023', 'latestUpdate': 1692388800378, 'latestVolume': 61172150, 'low': 171.96, 'lowSource':

In [57]:
def chunks(lst, n):
    """
    Yield successive n-sized chunks from lst.
    """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [64]:
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range (0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    vals = []
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns),
            ignore_index=True,
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,119.25,35223566297,N/A
1,AAL,15.04,9826571037,N/A
2,AAP,69.83,4150964185,N/A
3,AAPL,174.49,2728017141680,N/A
4,ABBV,150.14,265004108535,N/A
...,...,...,...,...
496,YUM,129.12,36180880603,N/A
497,ZBH,119.18,24904345490,N/A
498,ZBRA,271.98,13963008241,N/A
499,ZION,34.73,5145080469,N/A


In [60]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print('Please enter an integer')
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio: 10000000


In [61]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,119.25,35223566297,167
1,AAL,15.04,9826571037,1327
2,AAP,69.83,4150964185,285
3,AAPL,174.49,2728017141680,114
4,ABBV,150.14,265004108535,132
...,...,...,...,...
496,YUM,129.12,36180880603,154
497,ZBH,119.18,24904345490,167
498,ZBRA,271.98,13963008241,73
499,ZION,34.73,5145080469,574


In [62]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')

final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
writer.save()